In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

%env OMP_NUM_THREADS=20
import warnings
warnings.filterwarnings("error")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: OMP_NUM_THREADS=20


In [35]:
import numpy as np
import matplotlib.pyplot as plt

In [36]:
from collections import defaultdict

def δ(α, β):
    return np.kron(α, β)

def cov_2pt(signal_dict, noise_dict, bin_left, bin_right=None, 
            f_sky=0.6, N_splits=2,
            instruments=('Planck', 'Planck', 'Planck', 'Planck'), 
            observables=('T', 'T', 'T', 'T'), 
            seasons=(0,0,0,0),
            verbose=True):
    """Compute a covariance matrix for a 2-pt correlation function.
    
    This is an implementation of equation B2 in arXiv:1610.02360.
    
    signal_dict : dict
        Must have tuple keys for all two-point (T/E/B/κ) pairs
        i.e. a key could be ('T', 'E').
    noise_dict : dict
        Must have tuple keys in the format
            (season, instrument, season, instrument, T/E/B/κ, T/E/B/κ)
        If you have only one season, set every season value to 0.
    """
    
    # if bin_right is not specified, bin_right[n] = bin_left[n+1]
    if bin_right == None: 
        Δbin = bin_left[-1] - bin_left[-2] # last bin-width
        bin_right = np.hstack((np.array(bin_left[1:]), [bin_left[-1]+Δbin]))
        
    A, B, C, D = instruments
    W, X, Y, Z = observables
    α, β, γ, τ = seasons
    
    N_s = N_splits
    n_b = len(bin_left)
    
    # assume zero signal and infinite noise for non-specified terms
    S_b = defaultdict(lambda:0.0, signal_dict)
    N_b = defaultdict(lambda:np.inf, noise_dict)
    
    ν_b = np.zeros(n_b)
    for i, (bl, br) in enumerate(zip(bin_left, bin_right)):
        ν_b[i] = np.sum(2 * np.arange(bl, br) + 1) * f_sky
    
    # deliberately not PEP8, I think this is more readable
    sum1 = S_b[W,Y] * S_b[X,Z] + S_b[W,Z] * S_b[X,Y]
    
    sum2 = (
        S_b[W,Y] * δ(β,τ) * N_b[β,B,τ,D,X,Z] + 
        S_b[X,Z] * δ(α,γ) * N_b[α,A,γ,C,W,Y] +
        S_b[W,Z] * δ(β,γ) * N_b[β,B,γ,C,X,Y] + 
        S_b[X,Y] * δ(α,τ) * N_b[α,A,τ,D,W,Z]
    )
    
    sum3 = (
        δ(α,γ) * δ(β,τ) * N[α,A,γ,C,W,Y] * N[β,B,τ,D,X,Z] +
        δ(β,γ) * δ(α,τ) * N[α,A,τ,D,W,Z] * N[β,B,γ,C,X,Y]
    )
    
    prefactor_3 = (1/ν_b) * (
        (N_s**2 - N_s * (δ(α,β) + δ(γ,τ)) + N_s * δ(α,β) * δ(γ,τ)) /
        (N_s**4 - N_s**3 * (δ(α,β) + δ(γ,τ)) + N_s**2 * δ(α,β) * δ(γ,τ))
    )
    
    return (1/ν_b) * sum1 + (1/(N_s * ν_b)) * sum2 + prefactor_3 * sum3

In [28]:
cov_2pt({}, {}, np.arange(100))

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

--2019-09-30 08:27:02--  https://raw.githubusercontent.com/xzackli/cmb-mocks/master/notebooks/chains/class_bluebook.covmat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘raw.githubusercontent.com’
